# Populate `zip_code_dict.pickle`

^^ this is the one and only thing this file should do

In [3]:
from uszipcode import SearchEngine
from shapely.geometry import Polygon, Point
import networkx as nx
import random
import pickle

search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")
g_pickle = "01_cambridge-small.gpickle"
g = nx.read_gpickle(g_pickle)


In [4]:
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump({}, handle, protocol=pickle.HIGHEST_PROTOCOL) 


In [49]:
zip1 = set(['02121', '02124', '02119', '02120', '02467', '02446', '02149', '02150', '02148', '02129', '02145', '02127', '02210', '02111', '02118', '02110', '02143', '02163', '02139', '02138', '02134', '02163', '02139', '02143', '02134', '02138', '02143', '02141', '02139', '02145', '02142', '02143', '02141', '02139', '02145', '02142', '02142', '02116', '02114', '02111', '02134', '02446', '02139', '02115', '02215', '02134', '02446', '02139', '02163', '02115', '02143', '02163', '02139', '02138', '02134', '02134', '02163', '02139', '02446', '02138', '02138', '02140' '02116', '02142', '02141', '02142', '02114', '02129', '02139', '02142', '02141', '02116', '02199', '02143', '02141', '02145', '02139', '02142'])
zip2 = set(['02140', '02171', '02125', '02170', '02151', '02128', '02155'])
zip3 = set(['02445', '02130', '02131', '02464', '02108', '02122', '02453', '02176', '02108', '01906', '01905', '02180', '01901', '01908', '01902', '01904', '01880', '02122', '02130', '02131', '02445', '02108', '02169', '02152', '99999', '02445', '02130', '02108', '02131', '02122', '02464', '02445', '02130', '02131', '02108', '02122', '02464', '02465', '02130', '02445', '02131', '02122', '02108', '02464', '02026', '02186', '02445', '02130', '02131', '02464', '02460', '02494', '02468', '02445', '02130', '02108', '02131', '02130', '02445', '02131', '02122', '02136', '02108', '02461', '02122', '02474', '02478', '02476', '01890', '02130', '02445', '02131', '02108', '02130', '02445', '02445', '02130', '02445', '02130', '02131', '02472', '02458', '02109', '02113', '02203', '02132', '02135', '02126', '02144', '02459'])
zip_codes = zip1.union(zip2).union(zip3)

bad_zips = ['02130', '02131', '02122', '02464', '02108', '02445', '01902']
polygons = {}
zip_codes_copy = zip_codes.copy()
for z in zip_codes_copy:
            try: polygons[z] = Polygon(search.by_zipcode(z).polygon)
            except: 
                print(z)
                if z not in bad_zips:
                    bad_zips.append(z)
                zip_codes.remove(z)
print(zip_codes)

with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in zip_codes:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break
            
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)            
        

zip_code_dict
        
unprocessed = []
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    if node_id not in zip_code_dict:
        unprocessed.append(node_id)
print("Number still unprocessed: " + str(len(unprocessed)))
print("Add the following to the zip_codes set:")

random_unprocesseds = random.sample(unprocessed, 10)
unprocessed_data = []
for random_unprocessed in random_unprocesseds:
    zip_suggs = search.by_coordinates(g.nodes[random_unprocessed]['lat'], g.nodes[random_unprocessed]['lon'], returns=50)
    unprocessed_data.append(zip_suggs)
# print(unprocessed_data[0])
sugg = [z_data.zipcode for z_data in unprocessed_data[0] if z_data.zipcode not in zip_codes]
print(sugg)
print(str([x for x in sugg if x not in bad_zips]))

print(bad_zips)
# print(str(g.nodes[random_unprocessed]['lat']) + str(g.nodes[random_unprocessed]['lon']))

02130
02131
02122
02464
02108
02445
01902
{'02121', '02026', '02459', '02139', '02150', '02145', '02113', '02132', '01901', '01906', '02458', '02127', '02110', '02169', '02143', '02472', '02170', '02494', '02152', '01905', '02120', '02109', '02129', '02199', '02210', '02148', '02474', '02119', '02111', '01880', '02149', '02114', '02171', '02151', '02128', '02453', '02115', '0214002116', '02125', '02467', '02468', '02144', '02142', '02163', '01890', '02134', '01908', '02186', '02155', '02176', '02446', '02460', '99999', '02118', '02140', '02478', '02135', '02215', '02136', '02138', '02465', '02141', '02124', '02476', '01904', '02203', '02116', '02126', '02180', '02461'}
Number still unprocessed: 7811
Add the following to the zip_codes set:
['02130', '02445', '02131', '02122', '02108', '02464']
[]
['02130', '02131', '02122', '02464', '02108', '02445', '01902']


In [50]:
sugg

['02130', '02445', '02131', '02122', '02108', '02464']

In [55]:
def get_zip_code_dict():
    g = nx.read_gpickle("01_boston-area.gpickle")
    nodes_data_subset = g.nodes().data()
    search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")
    len_g = len(g)
    count = 1
    with open('zip_code_dict.pickle', 'rb') as handle:
        zip_code_dict = pickle.load(handle)

    for node_data in g.nodes().data():
        node_id = node_data[1]["id"]
        lon,lat = node_data[1]['lon'], node_data[1]['lat']
        if node_id not in zip_code_dict:
            count += 1
            zip_code_dict[node_id] = search.by_coordinates(lat, lon, returns=1)[0].zipcode
            print(zip_code_dict[node_id])
            print(str(len(zip_code_dict)) + "       "+ str(len(zip_code_dict)/len_g) + "        ...slow algo alert")


        if count % 25 == 0: 
            print("--> Dumping into zip_code_dict.pickle")
            with open('zip_code_dict_NEW.pickle', 'wb') as handle:
                pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        

    with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return zip_code_dict

In [ ]:
get_zip_code_dict()

02143
10771       0.11177874636778747        ...slow algo alert
02138
10772       0.11178912411789124        ...slow algo alert
02138
10773       0.11179950186799502        ...slow algo alert
02138
10774       0.1118098796180988        ...slow algo alert
02138
10775       0.11182025736820257        ...slow algo alert
02138
10776       0.11183063511830635        ...slow algo alert
02140
10777       0.11184101286841013        ...slow algo alert
02138
10778       0.1118513906185139        ...slow algo alert
02138
10779       0.11186176836861768        ...slow algo alert
02138
10780       0.11187214611872145        ...slow algo alert
02138
10781       0.11188252386882525        ...slow algo alert
02138
10782       0.11189290161892902        ...slow algo alert
02138
10783       0.1119032793690328        ...slow algo alert
02138
10784       0.11191365711913658        ...slow algo alert
02138
10785       0.11192403486924035        ...slow algo alert
02138
10786       0.11193441261934413      

02138
10897       0.11308634288086343        ...slow algo alert
02138
10898       0.1130967206309672        ...slow algo alert
02163
10899       0.11310709838107098        ...slow algo alert
02163
10900       0.11311747613117476        ...slow algo alert
02163
10901       0.11312785388127854        ...slow algo alert
02163
10902       0.11313823163138231        ...slow algo alert
02163
10903       0.11314860938148609        ...slow algo alert
02163
10904       0.11315898713158987        ...slow algo alert
02138
10905       0.11316936488169364        ...slow algo alert
02163
10906       0.11317974263179742        ...slow algo alert
02163
10907       0.11319012038190121        ...slow algo alert
02163
10908       0.11320049813200499        ...slow algo alert
02163
10909       0.11321087588210876        ...slow algo alert
02163
10910       0.11322125363221254        ...slow algo alert
02163
10911       0.11323163138231632        ...slow algo alert
02163
10912       0.1132420091324201     